In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

%matplotlib inline

### Carga de datos de entrenamiento y test

In [2]:
df = pd.read_csv('data/clean/train_clean_v2.csv')
df_targets = pd.read_csv('data/clean/train_labels.csv')
df.shape, df_targets.shape

((10795392, 27), (10795392, 24))

In [13]:
df = pd.read_csv('data/train_ver2.csv',
                 dtype={'sexo': str, 'age': str, 'ind_nuevo': str, 'indrel_1mes': str, 
                        'antiguedad': str, 'ult_fec_cli_lt': str, 'indext': str, 'conyuemp': str},
                 parse_dates=['fecha_dato', 'fecha_alta'])
df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [2]:
df_test = pd.read_csv('data/test_ver2.csv', dtype={'sexo': str, 'age': str, 'ind_nuevo': str,
                                                    'indrel_1mes': str, 'antiguedad': str, 'ult_fec_cli_lt': str,
                                                    'indext': str, 'conyuemp': str}, parse_dates=['fecha_dato', 'fecha_alta'])
df_test.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


## Preprocesado de los datos de test
En esta sección se le hace un preprocesado a los datos de test que consiste en:
* Eliminar columnas que tienen muchos datos vacíos
* Cambiar el tipo de datos de algunas columnas
* Rellenar los datos faltantes en ciertas filas, **no se pueden eliminar filas porque en la _submission_ de Kaggle se requieren de la totalidad de registros del dataset**
* Cambiar las variables categóricas a numéricas
* **Se guarda el dataset de test**

#### Chequeo de valores no definidos

In [9]:
df_test.isnull().sum().sort_values(ascending=False)

conyuemp                 929511
ult_fec_cli_1t           927932
renta                    227965
nomprov                    3996
cod_prov                   3996
segmento                   2248
canal_entrada              2081
tiprel_1mes                  23
indrel_1mes                  23
sexo                          5
indext                        0
indresi                       0
indfall                       0
tipodom                       0
ind_actividad_cliente         0
indrel                        0
antiguedad                    0
ind_nuevo                     0
fecha_alta                    0
age                           0
pais_residencia               0
ind_empleado                  0
ncodpers                      0
fecha_dato                    0
dtype: int64

#### Eliminación de columnas llenas de NaN

In [18]:
df_test.drop(labels=['conyuemp', 'ult_fec_cli_1t'], inplace=True, axis=1)

#### Se reemplazan algunos valores de renta a nan para poder sustituirlos por otro valores numéricos

In [8]:
df_test.loc[:, 'renta'].replace('         NA', np.nan, inplace=True)

In [16]:
renta_by_seg = df.groupby(['nomprov', 'segmento']).agg({'renta' : 'mean'})
renta_by_seg = renta_by_seg.loc[:, ['renta']].round(2)
renta_by_seg.iloc[:6]

renta
nomprov  segmento                     
ALAVA    01 - TOP             55271.07
         02 - PARTICULARES   145173.24
         03 - UNIVERSITARIO   89039.30
ALBACETE 01 - TOP             90440.96
         02 - PARTICULARES    85873.20
         03 - UNIVERSITARIO   81308.39

In [17]:
%%time
for i in renta_by_seg.index:
    renta_value = renta_by_seg.loc[i, 'renta']
    df_test.loc[(df_test['nomprov'] == i[0]) & (df_test['segmento'] == i[1]) & (df_test['renta'].isnull()), 'renta'] = renta_value

CPU times: user 49.1 s, sys: 20 ms, total: 49.1 s
Wall time: 49.1 s


Como se puede observar, aún quedan datos sin definir, estos datos serán reemplazados por valores nulos (por cero) porque no es posible reemplazarlos de manera acertiva

In [19]:
df_test.isnull().sum().sort_values(ascending=False)

renta                    4498
nomprov                  3996
cod_prov                 3996
segmento                 2248
canal_entrada            2081
tiprel_1mes                23
indrel_1mes                23
sexo                        5
indfall                     0
indext                      0
indresi                     0
tipodom                     0
ind_actividad_cliente       0
indrel                      0
antiguedad                  0
ind_nuevo                   0
fecha_alta                  0
age                         0
pais_residencia             0
ind_empleado                0
ncodpers                    0
fecha_dato                  0
dtype: int64

In [29]:
df_test.shape

(929615, 22)

#### Cambio de tipo de variable

In [30]:
#Cambio de tipo de variable
df_test.age = df_test.age.astype('int64')
df_test.ind_nuevo = df_test.ind_nuevo.astype('int64')
df_test.antiguedad = df_test.antiguedad.astype('int64')
df_test.indrel_1mes = df_test.indrel_1mes.astype('float64')

In [32]:
df_test.renta = df_test.renta.astype('float64')

#### Reemplazo de valores no definidos

In [36]:
df_test.indrel_1mes.replace(np.nan, 1, inplace=True)

In [37]:
df_test.sexo.replace(np.nan, 0, inplace=True)
df_test.tiprel_1mes.replace(np.nan, 0, inplace=True)
df_test.canal_entrada.replace(np.nan, 0, inplace=True)
df_test.cod_prov.replace(np.nan, 0, inplace=True)
df_test.nomprov.replace(np.nan, 0, inplace=True)
df_test.segmento.replace(np.nan, 0, inplace=True)
df_test.renta.replace(np.nan, 0, inplace=True)

#### División de fecha

In [38]:
df_test['fecha_dato_year'] = df_test.fecha_dato.dt.year
df_test['fecha_dato_month'] = df_test.fecha_dato.dt.month
df_test['fecha_dato_day'] = df_test.fecha_dato.dt.day

df_test['fecha_alta_year'] = df_test.fecha_alta.dt.year
df_test['fecha_alta_month'] = df_test.fecha_alta.dt.month
df_test['fecha_alta_day'] = df_test.fecha_alta.dt.day

In [41]:
cols = df_test.columns.tolist()
cols_ordered = cols[0:1] + cols[22:25] + cols[1:7] + cols[25:] + cols[7:22]

In [42]:
df_test = df_test[cols_ordered]
df_test.head()

,fecha_dato,fecha_dato_year,fecha_dato_month,fecha_dato_day,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,2016,6,28,15889,F,ES,V,56,1995-01-16,...,S,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,2016,6,28,1170544,N,ES,H,36,2013-08-28,...,S,N,KAT,N,1,3.0,ALICANTE,0,89980.13,02 - PARTICULARES
2,2016-06-28,2016,6,28,1170545,N,ES,V,22,2013-08-28,...,S,N,KHE,N,1,15.0,"CORUÑA, A",1,109258.62,03 - UNIVERSITARIO
3,2016-06-28,2016,6,28,1170547,N,ES,H,22,2013-08-28,...,S,N,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,2016,6,28,1170548,N,ES,H,22,2013-08-28,...,S,N,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


#### Cambio de variables categóricas a variables numéricas

In [43]:
cols = df_test.select_dtypes(['object']).columns
cols

Index(['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi',
       'indext', 'canal_entrada', 'indfall', 'nomprov', 'segmento'],
      dtype='object')

In [44]:
%%time
#Cambio de atributos categóricos a numéricos
cols = df_test.select_dtypes(['object']).columns

for col in cols:
    print(col, '¡LISTO!')
    attribute_vals = df_test[col].value_counts().index
    attribute_counts = np.arange(1, len(attribute_vals) + 1)
    df_test.replace(attribute_vals, attribute_counts, inplace=True)

ind_empleado ¡LISTO!
pais_residencia ¡LISTO!
sexo ¡LISTO!
tiprel_1mes ¡LISTO!
indresi ¡LISTO!
indext ¡LISTO!
canal_entrada ¡LISTO!
indfall ¡LISTO!
nomprov ¡LISTO!
segmento ¡LISTO!
CPU times: user 23min 56s, sys: 14 s, total: 24min 10s
Wall time: 24min 19s


In [48]:
df_test.shape

(929615, 28)

#### Exportar archivo

In [49]:
%%time
#Guardar
df_test.to_csv('data/clean/test_clean.csv', index=False)

CPU times: user 1min 9s, sys: 96 ms, total: 1min 10s
Wall time: 1min 10s
